In [296]:
import pandas as pd
from sklearn.mixture import BayesianGaussianMixture
from sklearn.decomposition import PCA
import numpy as np
import pickle
import json


In [317]:
# (game-changers, vct-international, vct-challengers)
LEAGUE = "game-changers"

# (2022, 2023, 2024)
YEAR = 2024

In [318]:
agg_df = pd.read_pickle(f'/content/{LEAGUE}-{YEAR}-agg.pkl')
df = pd.read_pickle(f'/content/{LEAGUE}-{YEAR}-df.pkl')

with open(f'/content/{LEAGUE}-{YEAR}-bggm.pkl', 'rb') as f:
    bgmm = pickle.load(f)

with open(f'/content/{LEAGUE}-{YEAR}-pca.pkl', 'rb') as f:
    pca = pickle.load(f)

with open(f'/content/players.json', 'r') as f:
    player_df = pd.DataFrame(json.load(f))

In [319]:
df.head()

,ROUND_NUMBER,OUTCOME,SIDE,KILLS,DEATHS,ASSISTS,COMBAT_SCORE,KILLS_STINGER,KILLS_BUCKY,KILLS_JUDGE,...,YORU_PICK_RATE,ISO_PICK_RATE,CLOVE_PICK_RATE,VYSE_PICK_RATE,DUELIST_PICK_RATE,INITIATOR_PICK_RATE,SENTINEL_PICK_RATE,CONTROLLER_PICK_RATE,id,metadata
0,1.0,1.0,-1.0,1.0,0.0,0.0,230.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,110196509462001155,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
1,1.0,1.0,-1.0,2.0,0.0,1.0,354.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,107025876838906695,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
2,1.0,1.0,-1.0,1.0,1.0,0.0,250.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,110196607873067874,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
3,1.0,1.0,-1.0,1.0,1.0,0.0,234.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,110196702294187829,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
4,1.0,1.0,-1.0,0.0,0.0,1.0,56.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,110196647859540239,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."


In [321]:
agg_df.drop(agg_df.index[:1], inplace=True)
agg_df.head()

,ROUND_NUMBER,SIDE,DEATHS,ASSISTS,KILLS_STINGER,KILLS_BUCKY,KILLS_JUDGE,KILLS_SPECTRE,KILLS_BULLDOG,KILLS_GUARDIAN,...,RAZE_PICK_RATE,REYNA_PICK_RATE,SAGE_PICK_RATE,SKYE_PICK_RATE,SOVA_PICK_RATE,VIPER_PICK_RATE,YORU_PICK_RATE,ISO_PICK_RATE,CLOVE_PICK_RATE,VYSE_PICK_RATE
id,,,,,,,,,,,,,,,,,,,,,
106371570922864110,0.740826,-0.450660,-0.809343,0.791967,-0.770870,-0.364931,-0.524846,0.371144,-0.929466,-1.237198,...,-0.518179,-0.246961,-0.276105,-0.450332,1.789826,-0.660963,-0.257586,-0.178576,-0.23074,0.0
106371572811742769,1.363058,-0.605352,-0.519918,0.184979,0.963865,-0.364931,-0.524846,-0.575765,0.676827,-0.845262,...,-0.518179,-0.246961,-0.276105,-0.450332,1.590935,1.619483,-0.257586,-0.178576,-0.23074,0.0
106371583096241717,0.132270,-0.251934,-0.674601,0.479166,0.836918,-0.364931,-0.524846,0.951732,-0.122018,-1.237198,...,-0.518179,-0.246961,-0.276105,-0.450332,1.778474,0.658654,-0.257586,-0.178576,-0.23074,0.0
106371585325121013,-0.753410,-3.218659,-0.484139,-1.665384,-0.770870,-0.364931,-0.524846,-1.061927,-1.451732,0.402176,...,-0.518179,-0.246961,-0.276105,-0.450332,1.890900,-0.660963,-0.257586,-0.178576,-0.23074,0.0
106371625243295242,0.740826,0.023797,0.362234,0.624839,-0.770870,-0.364931,-0.524846,1.203417,-0.940189,1.047520,...,-0.518179,-0.246961,-0.276105,2.330316,1.690366,-0.660963,-0.257586,-0.178576,-0.23074,0.0


In [322]:
target = '106371570922864110'
print(list(agg_df.loc[target].values))
print(bgmm.predict_proba(agg_df.loc[target].values.reshape(1,-1)))
print(bgmm.predict(agg_df.loc[target].values.reshape(1,-1)))
print(pca.transform(agg_df.loc[target].values.reshape(1,-1)))
player_df.loc[player_df['id'] == target].drop_duplicates('id')

[0.7408262791575616, -0.4506602420923629, -0.8093430974180001, 0.7919667222470715, -0.7708696985200681, -0.36493104904622786, -0.5248455724144154, 0.37114439411051664, -0.929465823523418, -1.2371975467085878, 1.908716305723392, -2.204851668785134, -0.40090768660019893, -0.568141424886023, -0.6900462449753433, -0.2198597432741583, -0.48908182448679466, -1.0993992574614118, -0.5250299267537103, -0.5174534748169473, -0.9469316735017599, -0.36476721698154585, -0.29047298713753417, -0.20281551440678144, -0.8866252848615821, -1.5615407629319455, -1.2665377429101485, -0.13682018290809, 0.027901959993732996, -0.2957734234600459, 1.937015259152714, -0.3942425755436335, -0.2034816850612795, -0.52142605164717, -0.2337182836758459, 2.3844802541743326, -0.49904686546433913, -0.23966040517718784, -0.5224143223105724, 0.0, -0.5648471555216513, -0.24728858272955603, -0.6252360499025587, -0.18093189121130213, -0.5181789279342509, -0.24696067188218532, -0.2761049032020882, -0.4503322475414043, 1.7898257

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
2089,106371570922864110,tayhuhu,Taynah,Faria,active,None,109993966808522027,2023-03-09T15:16:33Z,2024-07-16T02:10:01Z


In [323]:
X = agg_df.to_numpy()
pred = bgmm.predict(X)


In [332]:
import random
from collections import Counter
def weightedRandom(max):
    numDice = 5
    num = 0;
    for _ in range(numDice):
        num += random.random() * (max/numDice);
    return round(num);


dualists = list()

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 4:
        pc0 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][0]
        pc3 = -1 * pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][3]
        weight = 0.6 * pc0 + 0.3 * pc3
        if pc3 > 0:
            dualists.append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

print(weightedRandom(15))

dualists = sorted(dualists, key=lambda x: x[1], reverse=True)
pcaw_tot = sum([d[1] for d in dualists])
dualists = list(map(lambda x: (*x[0], (x[1]/pcaw_tot)*100), dualists))
for d in dualists:
    print(d)

print("-----")
d_counter = list()
for _ in range(1000):
    d_counter.append(dualists[weightedRandom(len(dualists)-1)][0])
duelist_exp = Counter(d_counter)
total_count = sum(duelist_exp.values())
duelist_pdf = { k: v/total_count for k,v in Counter(d_counter).items()}
duelist_pdf = {k:v for k, v in sorted(duelist_pdf.items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(duelist_pdf, indent=4))

10
('106977489224907631', 'LunaFox', 23.338765492191925)
('107176773267035613', 'Frantastic', 20.18225039016904)
('112279753081852283', 'shiye', 18.560544924098192)
('110745687059117485', 'maripo', 15.529287350839324)
('112812064625999649', 'Kyujin', 11.277216561687071)
('106371709362814570', 'antg', 9.052467801589684)
('112829098135710904', 'PenEloPE', 8.871685960395718)
('107600789960120743', 'Fire', 6.346130566416777)
('110422444218066611', 'Asia', 4.892582231344067)
('107600796790972557', 'mmonch', 0.4211607581258137)
('108490286624740396', 'luiza', -1.3393079337138813)
('109993752630503080', 'babz', -4.9219291363710385)
('109881266466748964', 'Raquinuki', -6.076888600859628)
('111057577168991497', 'krenn', -6.133966365913056)
-----
{
    "107600789960120743": 0.22,
    "112829098135710904": 0.212,
    "106371709362814570": 0.182,
    "110422444218066611": 0.155,
    "107600796790972557": 0.082,
    "112812064625999649": 0.08,
    "110745687059117485": 0.033,
    "10849028662474039

In [331]:
sentinals = list()

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 3 or pred[i] == 0:
        pc0 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][0]
        pc1 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][1]
        pc2 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][2]
        pc23 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][23]
        if pc0 < 4 and pc1 < 0 and pc2 > 0 and pc1 < pc0:
            weight = abs(-0.75 - pc23)
            sentinals.append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))


sentinals = sorted(sentinals, key=lambda x: x[1])
print(sentinals)
pcaw_tot = sum([d[1] for d in sentinals])
pcaw_max = max([s[1] for s in sentinals])
sentinals = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), sentinals))
for s in sentinals:
    print(s)
print("-----")
total_count = sum([s[2] for s in sentinals])
sentinals_pdf = {s[0]: s[2]/total_count for s in sentinals}
sentinals_pdf = {k:v for k, v in sorted(sentinals_pdf.items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(sentinals_pdf, indent=4))

[(['112748454218528350', 'GGonyaK'], 0.7893982370962607)]
('112748454218528350', 'GGonyaK', 0.0)
-----
{
    "112748454218528350": NaN
}


<ipython-input-331-2367a1757de2>:23: RuntimeWarning: invalid value encountered in scalar divide
  sentinals_pdf = {s[0]: s[2]/total_count for s in sentinals}


In [333]:
controller = list()

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 3:
        pc0 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][0]
        pc2 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][2]
        pc12 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][12]

        if pc2 <= 0:
            weight = abs(-0.25 - pc12)
            controller.append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))


controller = sorted(controller, key=lambda x: x[1])[:(len(controller)//2)]
pcaw_tot = sum([d[1] for d in controller])
pcaw_max = max([s[1] for s in controller])
controller = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), controller))
for s in sentinals:
    print(s)
print("-----")
total_count = sum([s[2] for s in controller])
controller_pdf = {s[0]: s[2]/total_count for s in controller}
controller_pdf = {k:v for k, v in sorted(controller_pdf.items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(controller_pdf, indent=4))

('112748454218528350', 'GGonyaK', 0.0)
-----
{
    "110196611566704749": 0.05796559989619855,
    "112184651889340927": 0.05707627129949759,
    "109080889947021472": 0.05639791147415325,
    "110765875517906795": 0.055383667099240884,
    "107600818318924231": 0.05409634751294575,
    "108748093853180323": 0.051857122412702746,
    "110422408105427408": 0.04993027679411635,
    "112002934600877930": 0.049814008185209,
    "110422485712945254": 0.04556140702621156,
    "111057427298594334": 0.04212772628160018,
    "108748099637064824": 0.03911375976344783,
    "112251772405825971": 0.03895611508819861,
    "106978644910376774": 0.038191542465578716,
    "107115833005547133": 0.0381179006568435,
    "106977394394708775": 0.03790847851313976,
    "112002780261921756": 0.03279730742105991,
    "107186009533585182": 0.032129557776544235,
    "107910753715863001": 0.03149557894714646,
    "106371664767204891": 0.029993784369845792,
    "110196702294187829": 0.0262219056624916,
    "1110077

In [334]:
initiator = list()

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 1:
        pc1 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][1]
        pc3 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][3]
        if pc1 >= 1.5:
            weight = abs(-1.25 - pc3)

            initiator.append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

initiator = sorted(initiator, key=lambda x: x[1])[:(len(initiator)//2)]
pcaw_tot = sum([d[1] for d in initiator])
pcaw_max = max([s[1] for s in initiator])
initiator = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), initiator))
for s in initiator:
    print(s)
print("-----")
total_count = sum([s[2] for s in initiator])
initiator_pdf = {s[0]: s[2]/total_count for s in initiator}
initiator_pdf = {k:v for k, v in sorted(initiator_pdf.items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(initiator_pdf, indent=4))

('112251569237859232', 'Misty', 7.081857765639551)
('108825689422016221', 'pigruto', 6.887094750994638)
('108449803223979248', 'wanheda', 6.886326527627388)
('111057427102102746', 'daf', 6.292582877258717)
('112251579675656496', 'Fer', 6.0920098509593315)
('110422481125123791', 'Hotaru', 5.841724724717463)
('108825685579240424', 'Felce ', 5.4559889325545115)
('112439951898540457', 'HellenCoco', 5.444690030068074)
('110169401198538681', 'panday', 5.049711390225371)
('112439952955196863', 'GulGul', 4.866616634452526)
('112439959768824239', 'SANGSIXK', 4.853610490065618)
('108076828770094503', 'pannyx', 4.799773303482039)
('112184635632773037', 'elfie', 4.676939321243764)
('107910753232568824', 'almndra', 4.471678379481321)
('110196517182501953', 'Desi', 4.171745361477323)
('110490591335960673', 'Sampphire', 3.8957049147729212)
('112273924591945659', 'Saab', 3.8031077475521853)
('112829096887354461', 'ran', 3.7785999373723365)
('112793696183610625', 'ayumi', 2.6011495845739097)
('11218467

In [328]:
target = '106371570922864110'
stripped_meta_df = df.loc[:, ~df.columns.isin(['metadata'])]
agg_plus_df = stripped_meta_df.groupby('id').agg({
    'ROUND_NUMBER': 'max',
    **{col: 'mean' for col in stripped_meta_df.columns if col not in ['id', 'ROUND_NUMBER']}
})
agg_plus_df.drop(agg_plus_df.index[:1], inplace=True)
agg_plus_df['MAX_PICK_RATE'] = agg_plus_df[['CONTROLLER_PICK_RATE', 'SENTINEL_PICK_RATE', 'INITIATOR_PICK_RATE', 'DUELIST_PICK_RATE']].max(axis=1)
agg_plus_df.head()
# agg_plus_df.loc[target, ['CONTROLLER_PICK_RATE', 'SENTINEL_PICK_RATE', 'INITIATOR_PICK_RATE', 'DUELIST_PICK_RATE']]
print(agg_plus_df.loc[target, ['MAX_PICK_RATE']])
player_df.loc[player_df['id'] == target].drop_duplicates('id')

MAX_PICK_RATE    1.0
Name: 106371570922864110, dtype: float64


,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
2089,106371570922864110,tayhuhu,Taynah,Faria,active,None,109993966808522027,2023-03-09T15:16:33Z,2024-07-16T02:10:01Z


In [335]:
flex_pool = agg_plus_df.loc[agg_plus_df['MAX_PICK_RATE'] <= 0.65, ['MAX_PICK_RATE']].sort_values(by='MAX_PICK_RATE', ascending=False).to_dict()['MAX_PICK_RATE']
flex_pool_map = player_df.loc[player_df['id'].isin(list(flex_pool.keys())), ['id', 'handle']].drop_duplicates('id').set_index('id').to_dict()['handle']

flex = list()
for i, id in enumerate(list(flex_pool_map.keys())):
    pc23 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][23]
    if pc1 >= 1.5:
        weight = abs(-0.9 - pc23)

        flex.append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

flex = sorted(flex, key=lambda x: x[1])[:(len(flex)//3)]
pcaw_tot = sum([f[1] for f in flex])
pcaw_max = max([f[1] for f in flex])
flex = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), flex))
for f in flex:
    print(f)
print("-----")
total_count = sum([f[2] for f in flex])
flex_pdf = {f[0]: f[2]/total_count for f in flex}
flex_pdf = {k:v for k, v in sorted(flex_pdf.items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(flex_pdf, indent=4))

('108748092607804521', 'HANE', 6.382432373713151)
('108076828770094503', 'pannyx', 6.340924146743486)
('106371585325121013', 'shizue', 6.107268753566259)
('110886997489179547', 'Gelatycat', 5.416833740827733)
('106977394394708775', 'JAZZYK1NS', 5.171164808588765)
('112184651942097411', 'phi', 3.8824112070148713)
('111006824262781761', 'Safiaa', 3.7206754397988204)
('108550493299217085', 'kitkatwc', 3.6553145406136474)
('109881707265575753', 'nati', 2.087647356398886)
('112002794971466948', 'Kazler', 2.0809437009449367)
('108231581673551401', 'Toki', 2.0257724185178327)
('107691411227382221', 'Looxie', 1.7939169548203917)
('111067931077980593', 'meimeiJP', 1.786264434358343)
('110765875517906795', 'UNSTABLE', 1.5815455471248705)
('111057427601760570', 'onni', 1.5085505870370868)
('106371664767204891', 'isa1', 1.2433878655100186)
('112439959768824239', 'SANGSIXK', 0.9112583771971985)
('107282729940754554', 'schnellAE', 0.817565456969766)
('107605566581730030', 'Mcfly', 0.8149418577714026